In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
import re
import collections
import random
import math

/home/ziyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
pd.set_option('display.max_column', None)
train_input = pd.read_csv('train.csv')
test_input = pd.read_csv('test.csv')
print("train_input shape is: ", np.shape(train_input))
print("test_input shape is: ", np.shape(test_input))

train_input shape is:  (4743, 2)
test_input shape is:  (1701, 2)


In [3]:
def label_tweet(input_set):
    handle = input_set['handle']
    # put it into an array named label, where 0 represents HillaryClinton, 1 represents readDonaldTrump
    label = []
    for i in range(len(handle)):
        if handle[i] == "HillaryClinton":
            label.append(0)
        if handle[i] == "realDonaldTrump":
            label.append(1)
    label = np.asarray(label)
    return label

train_label = label_tweet(train_input)
print(train_label[:10])

[0 0 0 0 1 0 0 0 1 1]


In [4]:
train_corpus = train_input['tweet'].as_matrix()
print(np.shape(train_corpus))
print(train_corpus[:5])
test_corpus = test_input['tweet'].as_matrix()
print(np.shape(test_corpus))

(4743,)
[ 'The question in this election: Who can put the plans into action that will make your life better? https://t.co/XreEY9OicG'
 'Last night, Donald Trump said not paying taxes was "smart." You know what I call it? Unpatriotic. https://t.co/t0xmBfj7zF'
 "If we stand together, there's nothing we can't do. \n\nMake sure you're ready to vote: https://t.co/tTgeqxNqYm https://t.co/Q3Ymbb7UNy"
 "Both candidates were asked about how they'd confront racial injustice. Only one had a real answer. https://t.co/sjnEokckis"
 'Join me for a 3pm rally - tomorrow at the Mid-America Center in Council Bluffs, Iowa! Tickets:… https://t.co/dfzsbICiXc']
(1701,)


In [5]:
# Load the stopwords
stop_words = stopwords.words('english')
# 'https' seems useless, so I add it to stop_words
stop_words.append(u'https')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [177]:
# Tokenize
def tokenization(text):
    return [i for i in text]
#     tokens=[]
# #     for word in nltk.word_tokenize(text.decode('utf-8')):
#     for word in nltk.word_tokenize(text):
#         # skip all the websites, punctuations, pure digits
#         if not re.match('[//]', word) and re.search('[a-zA-Z]', word) and word.lower() not in stop_words:
#             tokens.append(word.lower())
#     return tokens

# Tokenize training set
train_corpus_tokenized = []
for i in train_corpus:
    train_corpus_tokenized.append(' '.join(tokenization(i)))
    
print(len(train_corpus_tokenized))
print(train_corpus_tokenized[:5])

# Tokenize testing set
test_corpus_tokenized = []
for i in test_corpus:
    test_corpus_tokenized.append(' '.join(tokenization(i)))

print(len(test_corpus_tokenized))
print(test_corpus_tokenized[:5])

4743
['T h e   q u e s t i o n   i n   t h i s   e l e c t i o n :   W h o   c a n   p u t   t h e   p l a n s   i n t o   a c t i o n   t h a t   w i l l   m a k e   y o u r   l i f e   b e t t e r ?   h t t p s : / / t . c o / X r e E Y 9 O i c G', 'L a s t   n i g h t ,   D o n a l d   T r u m p   s a i d   n o t   p a y i n g   t a x e s   w a s   " s m a r t . "   Y o u   k n o w   w h a t   I   c a l l   i t ?   U n p a t r i o t i c .   h t t p s : / / t . c o / t 0 x m B f j 7 z F', "I f   w e   s t a n d   t o g e t h e r ,   t h e r e ' s   n o t h i n g   w e   c a n ' t   d o .   \n \n M a k e   s u r e   y o u ' r e   r e a d y   t o   v o t e :   h t t p s : / / t . c o / t T g e q x N q Y m   h t t p s : / / t . c o / Q 3 Y m b b 7 U N y", "B o t h   c a n d i d a t e s   w e r e   a s k e d   a b o u t   h o w   t h e y ' d   c o n f r o n t   r a c i a l   i n j u s t i c e .   O n l y   o n e   h a d   a   r e a l   a n s w e r .   h t t p s : / / t . c o / s j n E o 

In [178]:
train_tokenized_word = []
for i in range(len(train_corpus_tokenized)):
    train_tokenized_word.append(tf.compat.as_str(train_corpus_tokenized[i]).split())
print(len(train_tokenized_word))
print(train_tokenized_word[:5])

vocabulary_size = 10000

test_tokenized_word = []
for i in range(len(test_corpus_tokenized)):
    test_tokenized_word.append(tf.compat.as_str(test_corpus_tokenized[i]).split())
print(len(test_tokenized_word))
print(test_tokenized_word[:5])

4743
[['T', 'h', 'e', 'q', 'u', 'e', 's', 't', 'i', 'o', 'n', 'i', 'n', 't', 'h', 'i', 's', 'e', 'l', 'e', 'c', 't', 'i', 'o', 'n', ':', 'W', 'h', 'o', 'c', 'a', 'n', 'p', 'u', 't', 't', 'h', 'e', 'p', 'l', 'a', 'n', 's', 'i', 'n', 't', 'o', 'a', 'c', 't', 'i', 'o', 'n', 't', 'h', 'a', 't', 'w', 'i', 'l', 'l', 'm', 'a', 'k', 'e', 'y', 'o', 'u', 'r', 'l', 'i', 'f', 'e', 'b', 'e', 't', 't', 'e', 'r', '?', 'h', 't', 't', 'p', 's', ':', '/', '/', 't', '.', 'c', 'o', '/', 'X', 'r', 'e', 'E', 'Y', '9', 'O', 'i', 'c', 'G'], ['L', 'a', 's', 't', 'n', 'i', 'g', 'h', 't', ',', 'D', 'o', 'n', 'a', 'l', 'd', 'T', 'r', 'u', 'm', 'p', 's', 'a', 'i', 'd', 'n', 'o', 't', 'p', 'a', 'y', 'i', 'n', 'g', 't', 'a', 'x', 'e', 's', 'w', 'a', 's', '"', 's', 'm', 'a', 'r', 't', '.', '"', 'Y', 'o', 'u', 'k', 'n', 'o', 'w', 'w', 'h', 'a', 't', 'I', 'c', 'a', 'l', 'l', 'i', 't', '?', 'U', 'n', 'p', 'a', 't', 'r', 'i', 'o', 't', 'i', 'c', '.', 'h', 't', 't', 'p', 's', ':', '/', '/', 't', '.', 'c', 'o', '/', 't', '

In [179]:
cnt = collections.Counter()
for i in range(len(train_tokenized_word)):
    for word in train_tokenized_word[i]:
        cnt[word] += 1

print(len(cnt))

138


In [180]:
def build_dataset(cnt, words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(cnt.most_common(n_words - 1))
#     print count[:20]
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = []
    data_all = []
    unk_count = 0
    for i in range(len(words)):
        inner_data = []
        for word in words[i]:
            index = dictionary.get(word, 0)
            if index == 0:  # dictionary['UNK']
                unk_count += 1
            inner_data.append(index)
            data_all.append(index)
        data.append(inner_data)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, data_all, count, dictionary, reversed_dictionary

train_x, data_all, count, dictionary, reverse_dictionary = build_dataset(cnt, train_tokenized_word, vocabulary_size)

In [181]:
test = []
for sentence in test_tokenized_word:
    cur = []
    for word in sentence:
        if(word in dictionary):
            cur.append(dictionary[word])
        else:
            cur.append(0)
    test.append([cur,[0, 0]])

print(len(test))
print(test[:5])
test_length = len(test)
print(test_length)

1701
[[[29, 3, 14, 10, 11, 5, 45, 2, 22, 1, 15, 3, 7, 1, 13, 7, 3, 14, 11, 3, 21, 33, 31, 6, 10, 10, 4, 7, 18, 29, 10, 6, 5, 2, 3, 5, 17, 31, 1, 7, 25, 6, 8, 6, 3, 5, 4, 5, 11, 12, 3, 15, 15, 4, 5, 11, 11, 14, 7, 6, 5, 19, 10, 4, 8, 2, 5, 6, 19, 9, 2, 45, 8, 11, 1, 22, 4, 2, 1, 8, 9, 3, 20, 1, 11, 2, 9, 4, 2, 8, 9, 1, 45, 8, 7, 1, 4, 11, 18, 2, 3, 22, 1, 3, 14, 7, 5, 1, 53, 2, 33, 42, 43, 23, 55, 32, 17], [0, 0]], [[23, 9, 6, 8, 1, 10, 1, 12, 2, 6, 3, 5, 6, 8, 2, 3, 3, 6, 15, 13, 3, 7, 2, 4, 5, 2, 2, 3, 8, 6, 2, 3, 14, 2, 17, 44, 3, 2, 3, 9, 2, 2, 13, 8, 24, 16, 16, 2, 17, 12, 3, 16, 2, 23, 19, 1, 64, 53, 34, 64, 60, 15, 4, 5, 11, 15, 4, 26, 1, 8, 14, 7, 1, 18, 3, 14, 45, 7, 1, 7, 1, 19, 6, 8, 2, 1, 7, 1, 11, 17, 46, 34, 4, 2, 6, 3, 5, 4, 10, 61, 3, 2, 1, 7, 40, 1, 19, 6, 8, 2, 7, 4, 2, 6, 3, 5, 36, 4, 18, 59, 31], [0, 0]], [[43, 5, 12, 1, 4, 19, 4, 6, 5, 30, 20, 1, 20, 6, 10, 10, 9, 4, 25, 1, 4, 19, 3, 25, 1, 7, 5, 15, 1, 5, 2, 3, 21, 30, 22, 18, 4, 5, 11, 21, 3, 7, 2, 9, 1, 13, 1, 3,

In [182]:
train_all = [[train_x[i], [train_label[i], 1-train_label[i]]] for i in range(0, len(train_x))]
# train_all = [[train_x[i], [train_label[i]]] for i in range(0, len(train_x))]

r_index = list(range(len(train_all)))
random.shuffle(r_index)
train = [train_all[i] for i in r_index[:int(len(r_index)*0.8)]]
valid = [train_all[i] for i in r_index[int(len(r_index)*0.8):]]

print(len(train))
print(train[:5])
print(len(valid))
print(valid[:5])
print(len(count))

3794
[[[17, 33, 29, 34, 34, 13, 3, 10, 10, 51, 14, 8, 2, 9, 6, 2, 66, 71, 84, 21, 3, 7, 23, 7, 14, 15, 13, 17, 28, 5, 2, 1, 7, 1, 8, 2, 6, 5, 19, 9, 3, 20, 15, 18, 5, 14, 15, 22, 1, 7, 8, 9, 4, 25, 1, 19, 3, 5, 1, 8, 3, 21, 4, 7, 14, 13, 8, 6, 5, 12, 1, 10, 6, 19, 9, 2, 20, 1, 6, 19, 9, 2, 37, 4, 7, 12, 3, 40, 14, 22, 6, 3, 9, 4, 8, 2, 14, 7, 5, 1, 11, 5, 4, 8, 2, 18, 17, 49, 3, 25, 1, 6, 2, 38], [1, 0]], [[39, 1, 9, 4, 25, 1, 2, 9, 1, 15, 3, 8, 2, 13, 7, 3, 11, 14, 12, 2, 6, 25, 1, 20, 3, 7, 26, 1, 7, 8, 6, 5, 2, 9, 1, 20, 3, 7, 10, 11, 17, 39, 1, 51, 14, 8, 2, 5, 1, 1, 11, 2, 3, 19, 6, 25, 1, 3, 14, 7, 13, 1, 3, 13, 10, 1, 4, 12, 9, 4, 5, 12, 1, 2, 3, 8, 14, 12, 12, 1, 1, 11, 24, 9, 2, 2, 13, 8, 24, 16, 16, 2, 17, 12, 3, 16, 27, 43, 34, 5, 65, 19, 22, 4, 12, 9], [0, 1]], [[35, 33, 31, 27, 40, 18, 11, 1, 7, 24, 39, 9, 6, 12, 9, 6, 8, 6, 2, 33, 7, 1, 4, 10, 36, 3, 5, 4, 10, 11, 23, 7, 14, 15, 13, 76, 27, 7, 1, 18, 3, 14, 13, 10, 4, 5, 5, 6, 5, 19, 3, 5, 19, 1, 2, 2, 6, 5, 19, 7, 6, 11,

In [183]:
class SimpleDataIterator():
    def __init__(self, df):
        self.df = df
        self.size = len(self.df)
        self.epochs = 0
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.df)
        self.cursor = 0

    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            print("SimpleDataIterator epoch : ", self.epochs)
            self.shuffle()
        res = self.df[self.cursor:self.cursor+n]
        self.cursor += n
        return res

In [184]:
data = SimpleDataIterator(valid)
d = data.next_batch(500)
print('Input sequences : ', d[:5])

Input sequences :  [[[35, 44, 1, 2, 3, 5, 23, 7, 14, 15, 13, 45, 8, 49, 6, 8, 2, 35, 1, 15, 4, 6, 10, 21, 7, 3, 15, 2, 9, 1, 40, 34, 29, 20, 4, 8, 5, 3, 2, 4, 14, 2, 9, 3, 7, 6, 58, 1, 11, 17, 28, 4, 15, 8, 1, 10, 21, 21, 14, 5, 11, 6, 5, 19, 15, 18, 12, 4, 15, 13, 4, 6, 19, 5, 38, 36, 3, 5, 3, 2, 13, 4, 18, 17, 41, 15, 4, 6, 10, 24, 9, 2, 2, 13, 8, 24, 16, 16, 2, 17, 12, 3, 16, 57, 9, 62, 39, 69, 32, 8, 53, 58, 66], [1, 0]], [[28, 5, 10, 6, 19, 9, 2, 3, 21, 33, 26, 14, 7, 2, 1, 6, 12, 9, 1, 5, 20, 4, 10, 11, 45, 8, 7, 1, 13, 3, 7, 2, 3, 5, 2, 9, 1, 23, 7, 14, 15, 13, 43, 7, 19, 4, 5, 6, 58, 4, 2, 6, 3, 5, 45, 8, 11, 4, 5, 19, 1, 7, 3, 14, 8, 2, 6, 1, 8, 30, 9, 1, 7, 1, 4, 7, 1, 54, 47, 64, 14, 1, 8, 2, 6, 3, 5, 8, 2, 9, 4, 2, 23, 7, 14, 15, 13, 5, 1, 1, 11, 8, 2, 3, 4, 5, 8, 20, 1, 7, 63, 5, 3, 20, 17], [0, 1]], [[29, 7, 3, 3, 26, 1, 11, 31, 6, 10, 10, 4, 7, 18, 29, 10, 6, 5, 2, 3, 5, 8, 4, 18, 8, 2, 9, 4, 2, 8, 9, 1, 19, 3, 2, 15, 3, 7, 1, 13, 7, 6, 15, 4, 7, 18, 25, 3, 2, 1, 8, 2, 9

In [185]:
class PaddedDataIterator(SimpleDataIterator):
    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            self.shuffle()
        res = self.df[self.cursor:self.cursor+n]
        self.cursor += n

        # Pad sequences with 0s so they are all the same length
        max_len = 0
        for row in res:
            if len(row[0]) > max_len:
                max_len = len(row[0])
        seqlen = np.array([max_len for i in range(len(res))])
        ret = []
        label = []
        for row in res:
            ret += [row[0] + [0]*(max_len-len(row[0]))]
            label.append(row[1][0])
        x = np.array(ret)
        y = np.array(label)

        return x, y, seqlen

In [186]:
data = PaddedDataIterator(train)
d = data.next_batch(3)
print('Input sequences\n', d[0])
print(d[0].shape)
print(d[1])
print(d[1].shape)
print(d[2])

Input sequences
 [[42  7  6 15  1  7  3 30 14  5 35  2  4 12  3 22  3 20 10 35 17 60  4  9
   3  7  4 30 56  3  1 27  7 13  4  6  3 17 34  4 11  4 15 99  8 64 14  1
  11  1 12  6  7 17 46 40 34 29  6  5 29 49 41  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [37  3  7  1  2  9  4  5 69 47 15  6 10 10  6  3  5 27 15  1  7  6 12  4
   5  8  4  7  1  5 45  2  7  1 19  6  8  2  1  7  1 11  2  3 25  3  2  1
  17 39  1  9  4 25  1  2  3 11  3 22  1  2  2  1  7 38 44  1  2  7  1 19
   6  8  2  1  7  1 11 77  4 15 13 75  8 13  7  1  4 11  2  9  1 20  3  7
  11 24  9  2  2 13  8 24 16 16  2 17 12  3 16 28  3 22 52 68 42 72 69 51
  25]
 [23  9  4  5 26 18  3 14 34  1 20  2 38  9  2  2 13  8 24 16 16  2 17 12
   3 16 27 12 19 47  9 37 25  7 13  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  

In [187]:
batch_size = 256

In [188]:
def align(data):
    print(len(data))
    max_len = 0
    for row in data:
        if len(row[0]) > max_len:
            max_len = len(row[0])
    print(max_len)
    ret = []
    label = []
    for row in data:
        ret += [row[0] + [0]*(max_len-len(row[0]))]
        label.append(row[1][0])
    x = np.array(ret)
    y = np.array(label)
    seq_len = np.array([max_len for i in data])
    
    return x, y, seq_len

# def xx():
#     max_len = 0
#     for row in res:
#         if len(row[0]) > max_len:
#             max_len = len(row[0])
#     seqlen = np.array([max_len for i in range(len(res))])
#     ret = []
#     label = []
#     for row in res:
#         ret += [row[0] + [0]*(max_len-len(row[0]))]
#         label.append(row[1][0])
#     x = np.array(ret)
#     y = np.array(label)

# test_length = 1701
print(test_length)
print(batch_size)
print(test_length/batch_size)

test_list = []
test_addlen = test
test_addlen.extend(test[0:batch_size])
# for i in range(test_length/batch_size+1):
for i in range(math.ceil(test_length/batch_size)):
    print(i)
    x, y, seq_len = align(test[i*batch_size:(i+1)*batch_size])
    print(x.shape, ", ", y.shape, ", ", seq_len.shape)
#     print(y.shape)
#     print(seq_len.shape)
    test_list.append([x, y, seq_len])

print("=====")
print(len(test_list[0]))
    

# test_align, max_len = align(test)
# print(len(test_align))

1701
256
6.64453125
0
256
126
(256, 126) ,  (256,) ,  (256,)
1
256
129
(256, 129) ,  (256,) ,  (256,)
2
256
125
(256, 125) ,  (256,) ,  (256,)
3
256
131
(256, 131) ,  (256,) ,  (256,)
4
256
126
(256, 126) ,  (256,) ,  (256,)
5
256
128
(256, 128) ,  (256,) ,  (256,)
6
256
130
(256, 130) ,  (256,) ,  (256,)
=====
3


In [192]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

NUM_LAYERS=2

def gruCell(state_size):
#     lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, state_is_tuple=True)
    gru_cell = tf.nn.rnn_cell.GRUCell(state_size)
    return gru_cell
    
def build_graph(
    vocab_size = len(dictionary),
    state_size = 64,
    batch_size = 256,
    num_classes = 2):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
#     keep_prob = tf.constant(1.0)
    print("====",x.shape)
#     print("====",y.shape)
#     print("====",seqlen.shape)

    # Embedding layer
#     embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
#     rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    # RNN
    cell = tf.nn.rnn_cell.GRUCell(state_size)
#     cell = tf.nn.rnn_cell.BasicLSTMCell(state_size)
#     cell = tf.contrib.rnn.MultiRNNCell([gruCell(state_size) for _ in range(NUM_LAYERS)])
    init_state = tf.get_variable('gruCell', [1, state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, x, sequence_length=seqlen,
                                                 initial_state=init_state)

    # Add dropout, as the model otherwise quickly overfits
#     rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)

    """
    Obtain the last relevant output. The best approach in the future will be to use:

        last_rnn_output = tf.gather_nd(rnn_outputs, tf.pack([tf.range(batch_size), seqlen-1], axis=1))

    which is the Tensorflow equivalent of numpy's rnn_outputs[range(30), seqlen-1, :], but the
    gradient for this op has not been implemented as of this writing.

    The below solution works, but throws a UserWarning re: the gradient.
    """
    idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)

    # Softmax layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(last_rnn_output, W) + b
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
#     print(correct.shape)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
#     print(accuracy.shape)

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
#         'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }

In [193]:
def train_graph(graph, batch_size = 256, num_epochs = 150, iterator = PaddedDataIterator):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        tr = iterator(train)
        tv = iterator(valid)
#         te = iterator(test)

        step, accuracy = 0, 0
        tr_losses, tv_losses = [], []
        current_epoch = 0
        while current_epoch < num_epochs:
            step += 1
            batch = tr.next_batch(batch_size)
#             print("----",batch[0].shape)
#             print("----",batch[1].shape)
#             print("----",batch[2].shape)
#             dropout_parameter=np.array(0.6)
#             print("----",dropout_parameter.shape)
#             feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: dropout_parameter}
            feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
            accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
            accuracy += accuracy_

            if tr.epochs > current_epoch:
                current_epoch += 1
                tr_losses.append(accuracy / step)
                step, accuracy = 0, 0

                #eval test set
                tv_epoch = tv.epochs
                while tv.epochs == tv_epoch:
                    step += 1
                    batch = tv.next_batch(batch_size)
                    feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                    accuracy_ = sess.run([g['accuracy']], feed_dict=feed)[0]
                    accuracy += accuracy_

                tv_losses.append(accuracy / step)
                step, accuracy = 0,0
                print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1], "- tv:", tv_losses[-1])
            
        predictions = []
        for te in test_list:
            feed = {g['x']: te[0], g['y']: te[1], g['seqlen']: te[2]}
            preds_, _ = sess.run([g['preds'], g['ts']], feed_dict=feed)
            print(len(preds_))
            predictions.extend(preds_)

    return tr_losses, tv_losses, predictions

In [194]:
g = build_graph()
tr_losses, tv_losses, predictions = train_graph(g)
print(len(predictions))

==== (256, ?)


ValueError: Shape (?, 256) must have rank at least 3

In [37]:
# print(predictions[:test_length])
print(predictions[:50])

[array([  9.99783456e-01,   2.16587825e-04], dtype=float32), array([  9.99985099e-01,   1.49040734e-05], dtype=float32), array([  4.22566445e-05,   9.99957800e-01], dtype=float32), array([  9.99928355e-01,   7.15992355e-05], dtype=float32), array([  2.82522287e-05,   9.99971747e-01], dtype=float32), array([  9.99980330e-01,   1.97034951e-05], dtype=float32), array([ 0.0166364 ,  0.98336363], dtype=float32), array([  9.99961138e-01,   3.88448861e-05], dtype=float32), array([  9.99980211e-01,   1.98258767e-05], dtype=float32), array([ 0.84958726,  0.15041277], dtype=float32), array([  9.99469578e-01,   5.30452642e-04], dtype=float32), array([  9.99907732e-01,   9.22226100e-05], dtype=float32), array([  9.99968529e-01,   3.14640856e-05], dtype=float32), array([ 0.91769451,  0.08230553], dtype=float32), array([ 0.61804193,  0.3819581 ], dtype=float32), array([ 0.01473562,  0.98526436], dtype=float32), array([  9.99910355e-01,   8.96546262e-05], dtype=float32), array([  1.98048274e-05,   9.

In [173]:
import csv
with open('csvtest4.csv', 'w') as csvfile:
    csvfile.write(",".join(['id', 'realDonaldTrump', 'HillaryClinton'])+"\n")
    data = []
    for i in range(test_length):
        csvfile.write(",".join([str(i), str(predictions[i][1]), str(predictions[i][0])])+"\n")
